In [ ]:
%run notebook_preamble.ipy

In [ ]:
import re

# A quick look at patents and trademarks


Patent and trademark data are available from the the Intellectual Property Office.

* Patent data: https://www.gov.uk/government/publications/ipo-patent-data
* Trademark data: https://www.gov.uk/government/publications/ipo-trade-mark-data-release




### Patents

A guide for analysing patent data: https://www.praxisauril.org.uk/sites/praxisunico.org.uk/files/IPO_The_patent_guide.pdf


In [ ]:
patents_raw = pd.read_excel('https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/734638/PatentopendataJuly2018.xlsx')

In [ ]:
import pandas as pd

In [ ]:
patents_raw.to_csv(f"{data_path}/external/patents.csv")

In [ ]:
patents_raw = patents_raw.copy()

In [ ]:
patents = patents_raw.copy()

patents.columns = [re.sub(' ','_',x).lower() for x in patents.columns]

print(patents.shape)

patents.head().T

In [ ]:
patents['applicant_country_code'].value_counts().head()

Note the collaborative patents

In [ ]:
patents.applicant_country.value_counts().head()

Already geocoded

In [ ]:
patents.applicant_name.value_counts().head()

In [ ]:
patents.applicant_name.loc[patents.applicant_country=='Scotland'].value_counts().head()

The applicants seem to be companies

### Dates

In [ ]:
patents['filling_year'] = [int(str(x).split('-')[0]) if not pd.isnull(x) else x for x in patents.filing_date]

In [ ]:
patents['publication_year'] = [int(str(x).split('-')[0]) if not pd.isnull(x) else x for x in patents.a_publication_date]

In [ ]:
patents.publication_year.value_counts().plot.bar(color='blue')

Why has the number of patents in the data declined since the 1980s?

In [ ]:
patents.loc[(patents.publication_year==2018)&(patents.applicant_country=='Scotland')].head()

### Trademarks

In [ ]:
trademarks = pd.read_csv(f'{data_path}/external/trademarks.txt',encoding='utf-16',delimiter='|',error_bad_lines=False)

In [ ]:
trademarks.head()

In [ ]:
trademarks.columns = [re.sub(' ','_',x).lower() for x in trademarks.columns]

In [ ]:
trademarks.shape

In [ ]:
trademarks.country.value_counts().head()

In [ ]:
trademarks.region.value_counts()[:10]

In [ ]:
trademarks.postcode.value_counts().head()

Easy to geocode via postcodes

In [ ]:
trademarks['year_published'] = [int(str(x).split('-')[0]) if not pd.isnull(x) else x for x in trademarks.published]

In [ ]:
trademarks_published_counts = trademarks.year_published.value_counts()

In [ ]:
trademarks_published_counts[sorted(trademarks_published_counts.index)].plot()

In [ ]:
trademarks.columns

### Encoding the trademarks

In [ ]:
nice_codes = pd.read_csv(f'{data_path}/external/nice_codes.csv').dropna(axis=0,subset=['Code.1']).iloc[:-1,:]

nice_codes

In [ ]:
nice_codes['class'] = ['class'+str(int(x[3:])) for x in nice_codes['Code.1']]

In [ ]:
nice_df = nice_codes[['class','Description']]

nice_lookup = {x:y for x,y in zip(nice_df['class'],nice_df['Description'])}

In [ ]:
trademarks['is_scotland'] = [x=='Scotland' for x in trademarks.region]

tr_classes = trademarks.loc[:,['class' in x for x in trademarks.columns]]

In [ ]:
sc_classes = tr_classes.groupby(trademarks['is_scotland']).mean().T

In [ ]:
sc_classes.index= [nice_lookup[x][:20] for x in sc_classes.index]

In [ ]:
sc_classes['ratio']= sc_classes[True]/sc_classes[False]

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))


sc_classes['ratio'].sort_values(ascending=False).plot.bar(ax=ax,color='blue',title='Trademarks: Ratio of Scotland vs UK share')

In [ ]:
trademarks.to_csv(f'../data/external/{today_str}_trademarks.csv')

In [ ]:
with open('../metadata/nice_lookup.json','w') as outfile:
    json.dump(nice_lookup,outfile)